In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=38021 sha256=f62132a68e4dab51c0a41decaa4ad86cc186b4c0f1802f4ef11883c435d5aa3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-vzr75jad/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures


In [14]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py
import pickle
from tensorflow import keras

#append repo folder to search path
#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import get_trained_rnn_model, evaluate_trained_rnn

In [23]:
#define hyper params for each model
model_dict = {1:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 0, 'activation':''},\
              2:{'n_grus':24, 'n_dense_pre':1, 'n_dense_post': 0, 'activation':'tanh'},\
              3:{'n_grus':24, 'n_dense_pre':2, 'n_dense_post': 0, 'activation':'tanh'},\
              4:{'n_grus':24, 'n_dense_pre':3, 'n_dense_post': 0, 'activation':'tanh'},\
              5:{'n_grus':24, 'n_dense_pre':1, 'n_dense_post': 0, 'activation':'relu'},\
              6:{'n_grus':24, 'n_dense_pre':2, 'n_dense_post': 0, 'activation':'relu'},\
              7:{'n_grus':24, 'n_dense_pre':3, 'n_dense_post': 0, 'activation':'relu'},\
              8:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 1, 'activation':'tanh'},\
              9:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 2, 'activation':'tanh'},\
              10:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 3, 'activation':'tanh'},\
              11:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 1, 'activation':'relu'},\
              12:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 2, 'activation':'relu'},\
              13:{'n_grus':24, 'n_dense_pre':0, 'n_dense_post': 3, 'activation':'relu'},\
              14:{'n_grus':24, 'n_dense_pre':2, 'n_dense_post': 2, 'activation':'tanh'},\
              15:{'n_grus':24, 'n_dense_pre':2, 'n_dense_post': 2, 'activation':'relu'},\
              }

In [6]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'

nsubjects = 5

#randomly-selected subjects to use as hold-out test data 
test_subjects = [17, 23,  7,  8,  3]

# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap

In [7]:
#intialize empty lists
feature_matrix_all = np.empty((0,0))
target_labels_all = np.empty((0,))
window_tstamps_all = np.empty((0,))
block_labels_all  = np.empty((0,))
subject_id_all = np.empty((0,))
block_count = 0

for subject_id in range(1,nsubjects+1):
    if subject_id not in test_subjects:
        subject_folder = os.path.join(data_folder,'%02d'%(subject_id))
        print('=======================')
        print(subject_folder)

        # Process data and get features 
        #get features across segments and corresponding info
        feature_matrix, target_labels, window_tstamps, \
        block_labels, series_labels = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                        win_size, step)

        #prevent repeat of block labels by increasing block count
        block_labels = block_labels+block_count
        block_count = np.max([block_count, np.max(block_labels)])

        # concatenate lists
        feature_matrix_all = np.vstack((feature_matrix_all,feature_matrix)) if feature_matrix_all.size else feature_matrix
        target_labels_all = np.hstack((target_labels_all,target_labels))
        window_tstamps_all = np.hstack((window_tstamps_all,window_tstamps))
        block_labels_all = np.hstack((block_labels_all,block_labels))
        subject_id_all = np.hstack((subject_id_all,np.ones((block_labels.size))*subject_id))

/content/drive/MyDrive/EMG_gestures/EMG_data/01
/content/drive/MyDrive/EMG_gestures/EMG_data/02
/content/drive/MyDrive/EMG_gestures/EMG_data/04
/content/drive/MyDrive/EMG_gestures/EMG_data/05


In [24]:
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_joint_data/RNN/'
figure_folder = '/content/drive/MyDrive/EMG_gestures/figures/training_history/xsubject_joint_data/RNN'


#RNN training args - all other arguments are the same
verbose = 0
epochs = 30
batch_size = 5
validation_split = 0.1
# experiment params
n_splits = 4
nsets_training = 10
nreps = 2

#excluded labels
exclude = [0,7]
#performance metrics
score_list = ['f1','accuracy']

model_id = 1
rep = 0
#for model_id in range(1,5+1):
np.random.seed(1)# Set seed for replicability
results_df = []
for rep in range(nreps):
    print('Model %d | Rep %d'%(model_id, rep+1))
    print('--True Data--')
    rep_results_df, train_history = rnn_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                    block_labels_all, exclude,\
                                                                                    model_dict = model_dict[model_id], score_list = score_list,\
                                                                                    n_splits = n_splits, nsets_training = nsets_training,\
                                                                                verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                validation_split = validation_split, mv = None, permute = False)
#     #add details and concatenate dataframe
    rep_results_df['Shuffled'] = False
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)


    #plot training history
    fig_title = 'RNN model %02d; rep %i'%(model_id,rep)
    fig_fn = os.path.join(figure_folder,'rnn_model_%02d_rep_%i_loss.png'%(model_id,rep))
    plot_training_history(train_history, fig_title,fig_fn)

    #repeat with shuffled data
    print('Model %d | Rep %d'%(model_id, rep+1))
    print('--Permuted Data--')
    rep_results_df, train_history = rnn_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                    block_labels_all, exclude,\
                                                                                    model_dict = model_dict[model_id], score_list = score_list,\
                                                                                    n_splits = n_splits, nsets_training = nsets_training,\
                                                                                    verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                    validation_split = validation_split, mv = None, permute = True)
    #add details and concatenate dataframe
    rep_results_df['Shuffled'] = True
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)

#concatenate all data frames
results_df = pd.concat(results_df,axis = 0)

# # #save results to file
# results_fn = 'model_%02d_results.h5'%(model_id)
# results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')


Model 1 | Rep 1
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 1 | Rep 1
--Permuted Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 1 | Rep 2
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate 

In [25]:
results_df.groupby(['Shuffled','Type']).mean()

Fold  f1_score  accuracy_score  Rep
Shuffled Type                                      
False    Test    2.5  0.495204        0.549026  1.5
         Train   2.5  0.755401        0.764294  1.5
True     Test    2.5  0.121553        0.153190  1.5
         Train   2.5  0.325169        0.338099  1.5

In [ ]:
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_joint_data/RNN/'
model_dir = '/content/drive/MyDrive/EMG_gestures/model_data/xsubject_joint_data/RNN/'
figure_dir = '/content/drive/MyDrive/EMG_gestures/figures/training_history/xsubject_joint_data/RNN/'

#network training args 
verbose = 0
epochs = 30
batch_size = 2
validation_split = 0.1
nreps = 10

exclude = [0,7]

score_list = ['f1','accuracy']#performance metrics

model_id = 2
results_df = []
np.random.seed(1)
for rep in range(nreps):

    
    figure_folder = os.path.join(figure_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(figure_folder):
        os.makedirs(figure_folder)
    model_folder = os.path.join(model_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(model_folder):
        os.makedirs(model_folder)
    
    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----True Data----')
    rep_results_df, train_history = rnn_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                block_labels_all, exclude,\
                                                                                model_dict = model_dict[model_id], score_list = score_list,\
                                                                                n_splits = n_splits, nsets_training = nsets_training,\
                                                                            verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                            validation_split = validation_split, mv = None, permute = False)
    #add details and concatenate dataframe
    rep_results_df['Shuffled'] = False
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)


    #plot training history
    fig_title = 'RNN model %02d; rep %i'%(model_id,rep)
    fig_fn = os.path.join(figure_folder,'rnn_model_%02d_rep_%i_loss.png'%(model_id,rep))
    plot_train_loss(train_history, fig_title,fig_fn)

    #save trained model
    model_fn = os.path.join(model_folder, 'trained_model_rep_%i_all_train_data.h5'%(rep))
    keras.models.save_model(trained_model, model_fn, save_format= 'h5')

    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----Permuted Data----')
    rep_results_df, a, b, c = rnn_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                block_labels_all, exclude,\
                                                                                model_dict = model_dict[model_id], score_list = score_list,\
                                                                                n_splits = n_splits, nsets_training = nsets_training,\
                                                                            verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                            validation_split = validation_split, mv = None, permute = True)
    
    #add details and concatenate dataframe
    rep_results_df['Shuffled'] = True
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)

results_df = pd.concat(results_df, axis = 0).reset_index()

#save scaler (outisde of rep loop b/c it will always be the same)
scaler_fn = 'trained_scaler_all_training_data.pkl'#
with open(os.path.join(model_dir,scaler_fn), "wb") as output_file:
    pickle.dump(scaler, output_file)